In [9]:
import pickle
import pandas as pd
import numpy as np
import re
import urllib.request
from tqdm import tqdm
import time

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

import speech_recognition as sr
import pyaudio
from konlpy.tag import Okt

from pydub import AudioSegment
import wave
import contextlib
import os


class Voice:
    def __init__(self, input_file):
        self.file = input_file
        self.r = sr.Recognizer()
        
        self.loaded_model = load_model("./all_best_model.h5")
        self.type1_model = load_model("type1_best_model.h5")
        self.type2_model = load_model("type2_best_model.h5")
        self.stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
        self.df = pd.read_csv("./csv파일/call_all.csv")
        self.df = self.df.dropna(how="any")
        self.train_data = self.df[:600]
        self.test_data = self.df[600:]
        
        self.train_data.drop_duplicates(subset=["text"], inplace=True)
        self.train_data["text"] = self.train_data["text"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
        self.train_data["text"] = self.train_data["text"].str.replace("^ +", "", regex=True)
        self.train_data["text"].replace("", np.nan, inplace=True)
        self.train_data = self.train_data.dropna(how="any")
        self.okt = Okt()
        
        self.X_train = []
        for sentence in tqdm(self.train_data["text"]):
            tokenized_sentence = self.okt.morphs(sentence, stem=True)
            stopwords_removed_sentence = [word for word in tokenized_sentence if not word in self.stopwords]
            self.X_train.append(stopwords_removed_sentence)
        
        self.test_data["text"] = self.test_data["text"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
        self.test_data["text"] = self.test_data["text"].str.replace("^ +", "", regex=True)
        self.test_data["text"].replace("", np.nan, inplace=True)
        self.test_data = self.test_data.dropna(how="any")
        
        self.X_test = []
        for sentence in tqdm(self.test_data["text"]):
            tokenized_sentence = self.okt.morphs(sentence, stem=True)
            stopwords_removed_sentence = [word for word in tokenized_sentence if not word in self.stopwords]
            self.X_test.append(stopwords_removed_sentence)
        
        self.tokenizer = Tokenizer(3278)
        self.tokenizer.fit_on_texts(self.X_train)
        self.X_train = self.tokenizer.texts_to_sequences(self.X_train)
        self.X_test = self.tokenizer.texts_to_sequences(self.X_test)
        self.y_train = np.array(self.train_data["phishing"])
        self.y_test = np.array(self.test_data["phishing"])
        self.X_train = pad_sequences(self.X_train, maxlen=800)
        self.X_test = pad_sequences(self.X_test, maxlen=800)
        
        self.cnt = 1
        self.type1_cnt = 1
        self.type2_cnt = 1
        self.text = ''
        self.export_cnt = 0
    
    def to_wav(self):
        try:
            if self.file[self.file.rfind('.')+1:] != 'wav':
                sound = AudioSegment.from_file(self.file) 
                self.file = self.file[:self.file.rfind('.')]+'.wav'
                sound.export(self.file, format="wav")
                self.export_cnt = 1
            
            with contextlib.closing(wave.open(self.file, 'r')) as f:
                frames = f.getnframes()
                rate = f.getframerate()
                duration = frames / float(rate)            
            self.duration_list = [30] * int(duration/30) + [round(duration%30)]            
        except:
            print('Error')
            
    def recognize(self):
        try:
            with sr.AudioFile(self.file) as source:
                for duration in self.duration_list:
                    self.r.adjust_for_ambient_noise(source, duration=0.5)
                    self.r.dynamic_energy_threshold = True
                    audio = self.r.record(source, duration=duration)
                    try:
                        self.text += self.r.recognize_google(audio_data=audio, language='ko-KR')
                        print('▶ 통화내역 : {}'.format(self.text))
                    except:
                        None             
            if self.export_cnt == 1:
                if os.path.exists(self.file):
                    os.remove(self.file)
                    
        except: 
            print('Error')
            
    def sentiment_predict(self):
        self.text = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", self.text)
        self.text = self.okt.morphs(self.text, stem=True)
        self.text = [word for word in self.text if not word in self.stopwords]
        encoded = self.tokenizer.texts_to_sequences([self.text])
        pad_new = pad_sequences(encoded, maxlen=800)
        score = float(self.loaded_model.predict(pad_new))
        
        if score > 0.5:
            print("{}% 확률로 보이스피싱입니다.".format(score * 100))
            score1 = float(self.type1_model.predict(pad_new))
            score2 = float(self.type2_model.predict(pad_new))
            
            if score1 > score2:
                print("{}% 확률로 대출사기형 보이스피싱입니다.".format(score1 * 100))
            else:
                print("{}% 확률로 기관사칭형 보이스피싱입니다.".format(score2 * 100))       
        else:
            print("{}% 확률로 보이스피싱이 아닙니다.".format((1 - score) * 100))
                        
    def result(self):
        self.to_wav()
        self.recognize()
        self.sentiment_predict()

In [13]:

file_path = "D:/1조프로젝트/Voice_Phishing/피싱파일/type2/기관사칭22.wav" # 파일 경로 담기
vp = Voice(file_path)
vp.result()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22840\187224728.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.train_data.drop_duplicates(subset=["text"], inplace=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22840\187224728.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.train_data["text"] = self.train_data["text"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22840\187224728.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

▶ 통화내역 : 군인과 사건이 저희 쪽으로 사건 접수가 되어서요 맥박이 확인 차 연락 드렸어요 맞으시죠네 혹시 남성인데 김영철이라는 사람 아침에 분이십니까 들어본 적도 없으시고요 아 제가 왜 김영철 관련해서 얘기 하면 자기가 현재
▶ 통화내역 : 군인과 사건이 저희 쪽으로 사건 접수가 되어서요 맥박이 확인 차 연락 드렸어요 맞으시죠네 혹시 남성인데 김영철이라는 사람 아침에 분이십니까 들어본 적도 없으시고요 아 제가 왜 김영철 관련해서 얘기 하면 자기가 현재신용카드가 됐고 통장 등의 문제가 되는 것이 농협 통장이 불법 통장으로 가게 되었어요 그래서 저희가 현장에서 직접 없으시고요
▶ 통화내역 : 군인과 사건이 저희 쪽으로 사건 접수가 되어서요 맥박이 확인 차 연락 드렸어요 맞으시죠네 혹시 남성인데 김영철이라는 사람 아침에 분이십니까 들어본 적도 없으시고요 아 제가 왜 김영철 관련해서 얘기 하면 자기가 현재신용카드가 됐고 통장 등의 문제가 되는 것이 농협 통장이 불법 통장으로 가게 되었어요 그래서 저희가 현장에서 직접 없으시고요숫자 좀 세 명은 좀 줄여야 될 것 같아요 돈이 2014년 7월 7일 경에 경기도 광명시 철산동 농협관 하나은행 충전이 법인 계좌로 가결되었습니다 저희가 있기 때문에 지금 괜찮아 자체를 동시에 국고로 환수를 일단은 작년 7월경에
▶ 통화내역 : 군인과 사건이 저희 쪽으로 사건 접수가 되어서요 맥박이 확인 차 연락 드렸어요 맞으시죠네 혹시 남성인데 김영철이라는 사람 아침에 분이십니까 들어본 적도 없으시고요 아 제가 왜 김영철 관련해서 얘기 하면 자기가 현재신용카드가 됐고 통장 등의 문제가 되는 것이 농협 통장이 불법 통장으로 가게 되었어요 그래서 저희가 현장에서 직접 없으시고요숫자 좀 세 명은 좀 줄여야 될 것 같아요 돈이 2014년 7월 7일 경에 경기도 광명시 철산동 농협관 하나은행 충전이 법인 계좌로 가결되었습니다 저희가 있기 때문에 지금 괜찮아 자체를 동시에 국고로 환수를 일단은 작년 7월경에방귀 막혀서 이제 총장실 사람 수도 있으니까